In [1]:
!pip install pathway bokeh panel --quiet

import pandas as pd
import pathway as pw
import datetime
import panel as pn
pn.extension()


In [2]:
# Load dataset
df = pd.read_csv("dataset.csv")

# Combine date and time into Timestamp
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

# Sort by time
df = df.sort_values('Timestamp').reset_index(drop=True)

# Map vehicle weights
veh_weight_map = {"car": 1.0, "bike": 0.8, "truck": 1.5 ,"cycle": 0.5}
df['veh_weight'] = df['VehicleType'].map(veh_weight_map).fillna(1.0)

# Select columns for streaming
stream_df = df[[
    "Timestamp", "SystemCodeNumber", "Capacity",
    "Occupancy", "IsSpecialDay", "veh_weight"
]]

# Save to CSV
stream_df.to_csv("parking_stream.csv", index=False)


In [3]:
class ParkingSchema(pw.Schema):
    Timestamp:         str
    SystemCodeNumber:  str
    Capacity:          float
    Occupancy:         float
    IsSpecialDay:      int
    veh_weight:        float

source = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=500  # Simulated rows/second
)

# Parse datetime
source = source.with_columns(
    t = source.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S")
)


In [4]:
alpha, beta = 5.0, 0.5

hourly_price = (
    source
    .windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        window_end = pw.this._pw_window_end,
        occupancy  = pw.reducers.max(pw.this.Occupancy),
        capacity   = pw.reducers.max(pw.this.Capacity),
        is_special = pw.reducers.max(pw.this.IsSpecialDay),
        veh_weight = pw.reducers.max(pw.this.veh_weight),
        system_id  = pw.reducers.max(pw.this.SystemCodeNumber),
    )
    .with_columns(
        Price = 10
              + alpha
              * (pw.this.occupancy / pw.this.capacity)
              * pw.this.veh_weight
              * (1 + beta * pw.this.is_special)
    )
    .select(
        SystemCodeNumber = pw.this.system_id,
        Timestamp        = pw.this.window_end,
        veh_weight       = pw.this.veh_weight,
        Price            = pw.this.Price
    )
)

hourly_price.debug(name="📈 Hourly Price Output")


Traceback (most recent call last):
 File "/usr/local/lib/python3.11/dist-packages/pyviz_comms/__init__.py", line 341, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.11/dist-packages/panel/viewable.py", line 500, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/callbacks.py", line 441, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/callbacks.py", line 418, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/callbacks.py", line 453, in invoke_with_curdoc
 return f()
 ^^^
 File "/usr/local/lib/python3.11/dist-packages/bokeh/util/callback_manager.py", line 174, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 561, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.11/dist-packages/panel/io/state.py", line 488, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 559, in _comm_change
 self._schedule_change(doc, comm)
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 541, in _schedule_change
 self._change_event(doc)
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 537, in _change_event
 self._process_events(events)
 File "/usr/local/lib/python3.11/dist-packages/panel/widgets/tables.py", line 1381, in _process_events
 return super()._process_events(events)
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 1453, in _process_events
 self._process_data(events.pop('data'))
 File "/usr/local/lib/python3.11/dist-packages/panel/widgets/tables.py", line 1471, in _process_data
 return super()._process_data(data)
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 1426, in _process_data
 self._update_column(col, converted)
 File "/usr/local/lib/python3.11/dist-packages/panel/widgets/tables.py", line 1786, in _update_column
 self.value.loc[index, column] = array
 ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^
 File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 911, in __setitem__
 iloc._setitem_with_indexer(indexer, value, self.name)
 File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1942, in _setitem_with_indexer
 self._setitem_with_indexer_split_path(indexer, value, name)
 File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1998, in _setitem_with_indexer_split_path
 raise ValueError(
ValueError: Must have equal len keys and value when setting with an iterable
 


 Traceback (most recent call last):
 File "/usr/local/lib/python3.11/dist-packages/pyviz_comms/__init__.py", line 341, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.11/dist-packages/panel/viewable.py", line 500, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/callbacks.py", line 441, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/callbacks.py", line 418, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.11/dist-packages/bokeh/document/callbacks.py", line 453, in invoke_with_curdoc
 return f()
 ^^^
 File "/usr/local/lib/python3.11/dist-packages/bokeh/util/callback_manager.py", line 174, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.11/dist-packages/panel/reactive.py", line 561, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.11/dist-packages/panel/io/state.py", line 488, in _handle_exception
 raise exception
 Fil

<pathway.Table schema={'SystemCodeNumber': <class 'str'>, 'Timestamp': <class 'pathway.internals.datetime_types.DateTimeNaive'>, 'veh_weight': <class 'float'>, 'Price': <class 'float'>} id_type=pathway.engine.Pointer[tuple[str, pathway.internals.datetime_types.DateTimeNaive, pathway.internals.datetime_types.DateTimeNaive], pathway.internals.datetime_types.DateTimeNaive, pathway.internals.datetime_types.DateTimeNaive, str]>

In [5]:
pw.io.csv.write(
    table=hourly_price,
    filename="hourly_price_output.csv"
)

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [6]:
import panel as pn
import bokeh.plotting as bp
from bokeh.models import ColumnDataSource

pn.extension()

# 🔁 Streaming plot function
def price_plotter(source: ColumnDataSource):
    fig = bp.figure(
        height=400,
        width=800,
        title="Real-Time Parking Price",
        x_axis_type="datetime",
        x_axis_label="Timestamp",
        y_axis_label="Price"
    )

    # Line + small scatter dots (same color)
    fig.line("Timestamp", "Price", source=source, line_width=2, color="navy")
    fig.scatter("Timestamp", "Price", source=source, size=4, color="navy")

    return fig

# 📡 Attach plot to your hourly_price Pathway table
viz = hourly_price.plot(price_plotter, sorting_col="Timestamp")

# 🔽 Display using Panel
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [7]:
pw.run()


Output()

In [8]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot, column
from bokeh.models import ColumnDataSource, HoverTool, Legend
from bokeh.palettes import Category10

# Enable Bokeh output in notebook/Colab
output_notebook()

# Load your Pathway output
df = pd.read_csv("hourly_price_output.csv")
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# ✅ Map known vehicle weights to types
weight_to_type = {
    0.5: "Cycle",
    0.8: "Bike",
    1.0: "Car",
    1.5: "Truck"
}
df["veh_bin"] = df["veh_weight"].map(weight_to_type)

# 🎨 Color and plot setup
palette = Category10[10]
color_map = {}
plots = []
legend_items = {}

# 🔁 Create one plot per vehicle type
for bin_name in ["Cycle", "Bike", "Car", "Truck"]:
    p = figure(
        width=500,
        height=350,
        title=f"Vehicle Type: {bin_name}",
        x_axis_type="datetime",
        x_axis_label="Time",
        y_axis_label="Price"
    )

    # Add hover tool
    p.add_tools(HoverTool(
        tooltips=[("Time", "@x{%F %H:%M}"), ("Price", "@y")],
        formatters={"@x": "datetime"},
        mode="vline"
    ))

    # Filter data for current vehicle type
    sub_df = df[df["veh_bin"] == bin_name]

    # Line per system
    for syscode, sys_df in sub_df.groupby("SystemCodeNumber"):
        color = color_map.setdefault(syscode, palette[len(color_map) % len(palette)])
        sys_df = sys_df.sort_values("Timestamp")

        source = ColumnDataSource(data={
            "x": sys_df["Timestamp"],
            "y": sys_df["Price"]
        })

        line = p.line("x", "y", source=source, line_width=2, color=color)

        # Only collect one legend entry per system
        if syscode not in legend_items:
            legend_items[syscode] = (f"Lot {syscode}", [line])

    plots.append(p)

# ✅ Create separate legend figure
legend_fig = figure(
    width=800,
    height=120,
    title="Legend: Parking Lot (SystemCodeNumber)",
    toolbar_location=None,
    outline_line_color=None,
    min_border=0,
    x_range=(0, 1),
    y_range=(0, 1)
)
legend_fig.axis.visible = False
legend_fig.grid.visible = False

# Add all legend items
legend = Legend(items=list(legend_items.values()), location="center", orientation="horizontal", label_text_font_size="9pt")
legend_fig.add_layout(legend, 'center')

# 📊 Layout: 2x2 grid of subplots + legend below
grid = gridplot([plots[i:i+2] for i in range(0, len(plots), 2)])
show(column(grid, legend_fig))
